## License Plate Detection using Fuzzy Join 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB

In [1]:
%pip install "evadb[vision,notebook]"
%pip install thefuzz
%pip install "protobuf==3.20.1"
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-3.20.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.3
    Uninstalling protobuf-4.23.3:
      Successfully uninstalled protobuf-4.23.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.56.0 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 3.20.1 which is incompatible.
ray 2.4.0 requires grpcio<=1.51.3,>=1.42.0; python_version >= "3.10" and sys_platform != "darwin", but you have grpcio 1.56.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


### Loading the images to EvaDB for analysis

In [2]:
# Download images
!wget -nc "https://www.dropbox.com/s/770stddqfl0psog/license.zip"
!unzip -n license.zip

cursor.query('DROP TABLE IF EXISTS MyImages;').df()

cursor.load("license/Car*.png", "MyImages", format="image").df()

File 'license.zip' already there; not retrieving.

Archive:  license.zip


,0
0,Number of loaded IMAGE: 7


### License Plate Recognition

In [3]:
cursor.query("""DROP UDF IF EXISTS OCRExtractor;""").df()

cursor.query("""DROP UDF IF EXISTS FuzzDistance;""").df()

cursor.create_udf("OCRExtractor", True, '../evadb/udfs/ocr_extractor.py').df()
cursor.create_udf("FuzzDistance", True, '../evadb/udfs/fuzzy_join.py').df()

06-24-2023 13:57:42 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_udf:0081] UDF OCRExtractor does not exist, therefore cannot be dropped.
06-24-2023 13:57:42 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_udf:0081] UDF FuzzDistance does not exist, therefore cannot be dropped.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,0
0,UDF FuzzDistance successfully added to the dat...


In [4]:
cursor.query(
    "CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));"
).df()

""


In [5]:
cursor.load("/home/jarulraj3/eva/tutorials/data.csv", "LicensePlateCSV",format="csv").df()

,CSV,Number of loaded frames
0,/home/jarulraj3/eva/tutorials/data.csv,5


In [6]:
query = cursor.table("MyImages")
query = query.select("OCRExtractor(data)")
query = query.filter("_row_id < 3")
print(query.df())

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
query = cursor.table("LicensePlateCSV")
query = query.select("*")

query.df()

,licenseplatecsv._row_id,licenseplatecsv.id,licenseplatecsv.label
0,1,1,KLG1CA2555
1,2,2,PGMN112
2,3,3,PRENUP
3,4,4,DZ17YXR
4,5,5,PUI8BES


### Run Fuzzy Join to match Detected License Plate against Local License Plate Database (csv)

In [ ]:
cursor.query("""
   SELECT * FROM MyImages 
       JOIN LATERAL OCRExtractor(data) AS T(a) 
       JOIN LicensePlateCSV B 
       ON FuzzDistance(T.a, B.label) > 50
       WHERE id < 3;
       """).df()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
